In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import pandas as pd

import plotly
import plotly.subplots as sp
import plotly.express as px
import plotly.graph_objs as go

import FinanceDataReader as fdr

In [3]:
code='005930'
raw_df = fdr.DataReader(code)
raw_df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1998-07-07,872,921,869,875,390230,NaN
1998-07-08,904,904,867,875,237715,0.000000
1998-07-09,884,884,856,856,244510,-0.021714
1998-07-10,856,865,823,842,309070,-0.016355
1998-07-11,828,841,824,829,193470,-0.015439
...,...,...,...,...,...,...
2022-09-16,55600,56400,55500,56200,13456503,0.003571
2022-09-19,56300,57000,56000,56400,12278653,0.003559
2022-09-20,56400,57000,55800,55800,14041465,-0.010638


In [36]:
def MinMaxScaler(data):
    # 정규화
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # 0으로 나누기 에러가 발생하지 않도록 매우 작은 값(1e-7)을 더해서 나눔
    return numerator / (denominator + 1e-7)

def predict_stock(code):
    raw_df = fdr.DataReader(code, '20180901')

    window_size = 10
    data_size = 5
#     company = raw_df.company.values[0]
    dfx = raw_df[['Open','High','Low','Volume', 'Close']]
#     dfx = raw_df[['Volume', 'Close']]
    dfx = MinMaxScaler(dfx)
    dfy = dfx[['Close']]

    x = dfx.values.tolist()
    y = dfy.values.tolist()

    data_x = []
    data_y = []
    for i in range(len(y) - window_size):
        _x = x[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
        _y = y[i + window_size]     # 다음 날 종가
        data_x.append(_x)
        data_y.append(_y)
    print(_x, "->", _y)

    train_size = int(len(data_y) * 0.7)
    train_x = np.array(data_x[0 : train_size])
    train_y = np.array(data_y[0 : train_size])

    test_size = len(data_y) - train_size
    test_x = np.array(data_x[train_size : len(data_x)])
    test_y = np.array(data_y[train_size : len(data_y)])

    # 모델 생성
    model = Sequential()
    model.add(LSTM(units=10, activation='relu', return_sequences=True, input_shape=(window_size, data_size)))
    model.add(Dropout(0.1))
    model.add(LSTM(units=10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.summary()

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(train_x, train_y, epochs=70, batch_size=30)
    pred_y = model.predict(test_x)
    pre_df = pd.DataFrame(test_y)
    pre_df = pre_df.rename(columns = {0 : 'real'})
    pre_df['predict'] = pred_y
    #print(pre_df)
    predict = round(float((raw_df.Close[len(raw_df)-1] * pred_y[-1]) / dfy.Close[len(dfy)-1]), 0)
#     print(f"Tomorrow's {company} price : {predict} KRW")
    print(f"내일 가격 예측 : {predict} KRW")

    '''real_fig = go.Scatter(x= pre_df.index, y=pre_df.real, name="Real", line=dict(color='blue'))
    pred_fig = go.Scatter(x= pre_df.index, y=pre_df.predict, name="Predict", line=dict(color='red'))
    layout = go.Layout(title='Stock Price Prediction Test', xaxis_title = 'Times', yaxis_title = 'Value', xaxis_rangeslider_visible=True)
    plotly_fig = go.Figure(data = [real_fig, pred_fig], layout = layout)
    plotly_fig.update_xaxes(nticks=10)
    plotly_fig.show()'''

#     pre_return = [company, predict]
    pre_return = [predict]

    return pre_return

predict_stock('005930')

[[0.3736991485328785, 0.3378378378372672, 0.38081671414932416, 0.0360485187564102, 0.36694677871079934], [0.3642384105953373, 0.32263513513459013, 0.3618233618226746, 0.1510870370757987, 0.34640522875752305], [0.3547776726577961, 0.31756756756703114, 0.3561253561246797, 0.15824844027878746, 0.33893557422905896], [0.369914853357862, 0.35304054053994416, 0.3789173789166592, 0.14854601379847346, 0.3856209150319596], [0.3547776726577961, 0.3310810810805218, 0.3656220322880045, 0.0945106339553482, 0.36134453781445125], [0.369914853357862, 0.3293918918913355, 0.36372269705533955, 0.0844972981611588, 0.34640522875752305], [0.3434247871327466, 0.31756756756703114, 0.35422602089201477, 0.10535517224366663, 0.3501400560217551], [0.3566698202453043, 0.32770270270214913, 0.36372269705533955, 0.0916432414634164, 0.35387488328598715], [0.3585619678328126, 0.32770270270214913, 0.35992402659000966, 0.11216500240292013, 0.342670401493291], [0.3396404919577301, 0.3023648648643541, 0.34472934472869, 0.08

2022-09-22 16:10:33.484481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 6s 202ms/step - loss: 0.0362
Epoch 2/70
24/24 [==============================] - 5s 202ms/step - loss: 0.0096
Epoch 3/70
24/24 [==============================] - 5s 216ms/step - loss: 0.0078
Epoch 4/70
24/24 [==============================] - 6s 237ms/step - loss: 0.0085
Epoch 5/70
24/24 [==============================] - 6s 232ms/step - loss: 0.0070
Epoch 6/70
24/24 [==============================] - 5s 213ms/step - loss: 0.0079
Epoch 7/70
24/24 [==============================] - 5s 213ms/step - loss: 0.0083
Epoch 8/70
24/24 [==============================] - 5s 221ms/step - loss: 0.0062
Epoch 9/70
24/24 [==============================] - 5s 223ms/step - loss: 0.0072
Epoch 10/70
24/24 [==============================] - 5s 200ms/step - loss: 0.0058
Epoch 11/70
24/24 [==============================] - 5s 205ms/step - loss: 0.0064
Epoch 12/70
24/24 [==============================] - 5s 200ms/step - loss: 0.0039
Epoch 13/70
24/24 [=================

2022-09-22 16:16:28.942137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 30ms/step
내일 가격 예측 : 56979.0 KRW


[56979.0]